**1. write the dual LP problem** 

**Given primal LP problem:**

\begin{align*}
\text{maximize} \quad & x_1 + 4x_2 + 2x_3 \\
\text{subject to} \quad & 5x_1 + 2x_2 + 2x_3 \leq 145 \\
& 4x_1 + 8x_2 - 8x_3 \leq 260 \\
& x_1 + x_2 + 4x_3 \leq 190 \\
& x \geq 0
\end{align*}

**Dual LP problem:**

\begin{align*}
\text{minimize} \quad & 145y_1 + 260y_2 + 190y_3 \\
\text{subject to} \quad & 5y_1 + 4y_2 + y_3 \geq 1 \\
& 2y_1 + 8y_2 + y_3 \geq 4 \\
& 2y_1 - 8y_2 + 4y_3 \geq 2 \\
& y \geq 0
\end{align*}

**Standard form Dual LP problem:**

\begin{align*}
\text{maximize} \quad & -145y_1 - 260y_2 - 190y_3 \\
\text{subject to} \quad & 5y_1 + 4y_2 + y_3 - s = 1 \\
& 2y_1 + 8y_2 + y_3 - t = 4 \\
& 2y_1 - 8y_2 + 4y_3 - u = 2 \\
& y \geq 0
\end{align*}

**2. verify that  $Q=(x_1,x_2,x_3)=(0,52.5,20)$ is a feasible solution for the primal**

In [8]:
""" I used the architecture of the code from E4basicLPcode.ipynb (Cf. GitHub repo ) 
and modified it to fit our new objective function, constraints, and coefficients.
I kept the same solver, and simplified the status to only return the two expected outcomes"""

from ortools.linear_solver import pywraplp

def create_solver():
    # Create the linear solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('SCIP solver not available.')
    return solver

def verify_feasibility_Q():
    solver = create_solver()

    # Variables x1, x2, x3
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    x3 = solver.NumVar(0, solver.infinity(), 'x3')

    # Objective Function: this is a feasibility check, so we can use a dummy objective.
    # Here we just minimize x1 which won't affect the feasibility.
    objective = solver.Objective()
    objective.SetCoefficient(x1, 1)
    objective.SetMinimization()

    # Constraints
    constraint1 = solver.Constraint(-solver.infinity(), 145)
    constraint1.SetCoefficient(x1, 5)
    constraint1.SetCoefficient(x2, 2)
    constraint1.SetCoefficient(x3, 2)

    constraint2 = solver.Constraint(-solver.infinity(), 260)
    constraint2.SetCoefficient(x1, 4)
    constraint2.SetCoefficient(x2, 8)
    constraint2.SetCoefficient(x3, -8)

    constraint3 = solver.Constraint(-solver.infinity(), 190)
    constraint3.SetCoefficient(x1, 1)
    constraint3.SetCoefficient(x2, 1)
    constraint3.SetCoefficient(x3, 4)

    # Set the proposed solution Q as a starting point for the solver
    solver.SetHint([x1, x2, x3], [0, 52.5, 20])

    # Solve the problem
    status = solver.Solve()

    # Check and print if the proposed solution Q is feasible
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('The solution Q is feasible.')
        print(f'x1 = {x1.solution_value()}, x2 = {x2.solution_value()}, x3 = {x3.solution_value()}')
    else:
        print('The solution Q is not feasible.')

# Call the function to verify the feasibility of Q
verify_feasibility_Q()


The solution Q is feasible.
x1 = 0.0, x2 = 52.5, x3 = 20.0


**3. Use Complementary Slackness to determine a candidate solution to the dual.**

To determine a candidate solution to the dual using Column Generation (CG) and display the simplex tableau at each iteration, I needed to implement a structure that can handle the simplex method and also interact with ortools. Using the structure given to us in class; 
https://notebook.community/infimath/optimization-method/Simplex%20Tableau%20in%20Python
I tried to use this tableau generation method with around my code. 



In [9]:
from __future__ import division
import numpy as np
from fractions import Fraction

class Tableau:
    def __init__(self, obj):
        self.obj = [1] + obj
        self.rows = []
        self.cons = []
        self.no_variables = len(obj)
        self.no_constraints = 0
        self.is_fraction = False # set True to output in fraction

    def add_constraint(self, expression, value):
        self.rows.append([0] + expression)
        self.cons.append(value)
        self.no_constraints += 1
        self.header_tableau = ["Basic"] + ["x"+str(i+1) for i in range(self.no_variables)] \
                                        + ["s"+str(i+1) for i in range(self.no_constraints)] \
                                        + ["Solution"]
        self.basic_variables = ["s"+str(i+1) for i in range(self.no_constraints)]

    def _pivot_column(self):
        lowest = min(self.obj[1:-1])
        if lowest >= 0:
            return -1
        return self.obj.index(lowest)

    def _pivot_row(self, col):
        ratios = []
        for i, row in enumerate(self.rows):
            element = row[col]
            if element <= 0:
                ratios.append(float('inf'))  # Avoid division by zero and negative ratios
            else:
                ratios.append(self.rows[i][-1] / element)
        return np.argmin(ratios)

    def display(self):
        if self.is_fraction:
            # Formatting the output in fraction
            fmt = '{:<8}' + ''.join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)]) \
                + ''.join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) + '{:>8}'
            print(fmt.format("Basic", *["x"+str(i+1) for i in range(self.no_variables)] + ["s"+str(i+1) for i in range(self.no_constraints)] + ["Sol."]))

            row_format = '{:<8}' + ''.join(['{:>8}'.format(str(Fraction(x).limit_denominator())) for x in self.obj[1:]])
            print(row_format.format("z", *[str(Fraction(x).limit_denominator()) for x in self.obj[1:]]))

            for i, row in enumerate(self.rows):
                print(row_format.format(self.basic_variables[i], *[str(Fraction(x).limit_denominator()) for x in row[1:]]))
        else:
            # Formatting the output in float with 2 decimal places
            fmt = '{:<8}' + ''.join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)]) \
                + ''.join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) + '{:>8}'
            print(fmt.format("Basic", *["x"+str(i+1) for i in range(self.no_variables)] + ["s"+str(i+1) for i in range(self.no_constraints)] + ["Sol."]))

            row_format = '{:<8}' + ''.join(['{:>8.2f}'.format(x) for x in self.obj[1:]])
            print(row_format.format("z", *[f'{x:.2f}' for x in self.obj[1:]]))

            for i, row in enumerate(self.rows):
                print(row_format.format(self.basic_variables[i], *[f'{x:.2f}' for x in row[1:]]))


    def _pivot(self, row, col):
        pivot_element = self.rows[row][col]
        self.rows[row] = [x / pivot_element for x in self.rows[row]]
        for r in range(len(self.rows)):
            if r != row:
                self.rows[r] = [orig - factor * updated for orig, factor, updated in zip(self.rows[r], self.rows[row], [self.rows[r][col]]*len(self.rows[row]))]
        self.obj = [z - factor * updated for z, factor, updated in zip(self.obj, self.rows[row], [self.obj[col]]*len(self.obj))]

    def _check(self):
        return min(self.obj[1:-1]) >= 0

    def solve(self):
        # Append slack variables and setup tableau
        for i in range(len(self.rows)):
            identity = [0]*self.no_constraints
            identity[i] = 1
            self.rows[i] += identity + [self.cons[i]]
            self.rows[i] = np.array(self.rows[i], dtype=float)
        self.obj = np.array(self.obj + [0]*self.no_constraints, dtype=float)

        self.display()
        while not self._check():
            col = self._pivot_column()
            if col == -1:
                break
            row = self._pivot_row(col)
            self._pivot(row, col)
            print('\nEntering Variable: ', self.header_tableau[col])
            print('Leaving Variable: ', self.basic_variables[row], '\n')
            self.basic_variables[row] = self.header_tableau[col]
            self.display()

if __name__ == '__main__':
    t = Tableau([-2, -3, -2])
    t.add_constraint


\begin{align*}
\text{maximize} \quad & x_1 + 4x_2 + 2x_3 \\
\text{subject to} \quad & 5x_1 + 2x_2 + 2x_3 \leq 145 \\
& 4x_1 + 8x_2 - 8x_3 \leq 260 \\
& x_1 + x_2 + 4x_3 \leq 190 \\
& x \geq 0
\end{align*}

In [10]:
from ortools.linear_solver import pywraplp

def solve_dual_problem():
    # Create the linear solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print('SCIP solver is not available.')
        return

    # Define variables for the dual problem
    # These will correspond to the constraints in the primal problem
    y1 = solver.NumVar(0, solver.infinity(), 'y1')
    y2 = solver.NumVar(0, solver.infinity(), 'y2')
    y3 = solver.NumVar(0, solver.infinity(), 'y3')

    # Objective function: Minimize 145*y1 + 260*y2 + 190*y3
    objective = solver.Objective()
    objective.SetCoefficient(y1, 145)
    objective.SetCoefficient(y2, 260)
    objective.SetCoefficient(y3, 190)
    objective.SetMinimization()

    # Constraint 1: 5*y1 + 4*y2 + y3 >= 1
    c1 = solver.Constraint(1, solver.infinity())
    c1.SetCoefficient(y1, 5)
    c1.SetCoefficient(y2, 4)
    c1.SetCoefficient(y3, 1)

    # Constraint 2: 2*y1 + 8*y2 + y3 >= 4
    c2 = solver.Constraint(4, solver.infinity())
    c2.SetCoefficient(y1, 2)
    c2.SetCoefficient(y2, 8)
    c2.SetCoefficient(y3, 1)

    # Constraint 3: 2*y1 - 8*y2 + 4*y3 >= 2
    c3 = solver.Constraint(2, solver.infinity())
    c3.SetCoefficient(y1, 2)
    c3.SetCoefficient(y2, -8)
    c3.SetCoefficient(y3, 4)

    # Solve the problem and print the results
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        print('y1 =', y1.solution_value())
        print('y2 =', y2.solution_value())
        print('y3 =', y3.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print('Solver status is', status)

# Run the dual problem solver
solve_dual_problem()


Optimal Solution Found:
y1 = 1.5
y2 = 0.12499999999999994
y3 = 0.0
Objective value = 250.00000000000003


**4. Is $Q$ the optimal solution for the primal problem?**